# Params for Simple_RNN

$params = cell\times input\_size + cell^2$

$cell^2$ means cells are fullly connected to themselves   meaning?


In [0]:
%tensorflow_version 1.x

In [27]:
# colab.research.google.com
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import numpy as np

'''
# labs.cognitiveclass.ai
import keras
from keras.layers import *
import numpy as np
'''

'\n# labs.cognitiveclass.ai\nimport keras\nfrom keras.layers import *\nimport numpy as np\n'

In [28]:
cell = 1
input_size = 2
time_steps = 1    

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.get_weights()

[array([[-0.26524425],
        [ 1.0596277 ]], dtype=float32), array([[-1.]], dtype=float32)]

In [0]:
def to_npa_list(a):
  return [np.array(i) for i in a]

model.set_weights(to_npa_list([[[2],[3]],[[5]]]))


In [31]:
x = np.array([1, 0])
x = x.reshape(1,1,2)
model.predict(x)

array([[2.]], dtype=float32)

# 2 timesteps

In [32]:
cell = 1
input_size = 2
time_steps = 2    

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_5 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.set_weights(to_npa_list([[[2],[3]],[[5]]]))

In [34]:
x = np.array([1, 0, 0, 0])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [0 0]]]


array([[10.]], dtype=float32)

In [35]:
x = np.array([1, 0, 1, 0])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [1 0]]]


array([[12.]], dtype=float32)

In [36]:
x = np.array([1, 0, 0, 1])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [0 1]]]


array([[13.]], dtype=float32)

# 3 timesteps

In [37]:
cell = 1
input_size = 2
time_steps = 3    

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_6 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.set_weights(to_npa_list([[[2],[3]],[[5]]]))

In [39]:
x = np.array([1, 0, 0, 1, 7, 11])
x = x.reshape(1,3,2)
print(x)
model.predict(x) == 2 * 7 + 3 * 11 + 5 * 13

[[[ 1  0]
  [ 0  1]
  [ 7 11]]]


array([[ True]])

# Prepare Training Data

In [40]:
BIT = 4

def get_bit(n, bit):
    a = []
    for i in range(bit):
        a.append(n & 1)
        n >>= 1
    return a
  
for i in range(4):
    print(get_bit(i, BIT))


[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[1, 1, 0, 0]


In [0]:
time_steps = BIT

x = []
y = []
for i in range(2**time_steps):
    for j in range(2**time_steps):
        i_b = get_bit(i, time_steps)
        j_b = get_bit(j, time_steps)
        x.append([[i_b[i], j_b[i]] for i in range(time_steps)])
        y.append([get_bit(i+j, time_steps)[-1]])
    

In [42]:
x = np.array(x)
x.shape

(256, 4, 2)

In [43]:
y = np.array(y)
y.shape

(256, 1)

# Train

In [44]:
cell = 8
input_size = 2
time_steps = BIT    

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #LSTM(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        Dense(1, activation='relu', use_bias=False)    
])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_7 (SimpleRNN)     (None, 8)                 80        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8         
Total params: 88
Trainable params: 88
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', # tf.keras.optimizers.SGD(lr=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
history = model.fit(x, y, epochs=2000, verbose=0)    
print(history.history.keys())

dict_keys(['loss', 'acc'])


In [47]:
print(history.history['loss'][-1])
print(history.history['acc'][-1])

0.00020380895693961065
1.0


# Test

In [48]:
a = model.predict_classes(x)
all(a == y)

True

In [0]:
def add(a, b):
  a = get_bit(a, BIT)
  b = get_bit(b, BIT)
  x = [[a[i], b[i]] for i in range(BIT)]
  x = [[x]]
  return model.predict_classes(x)[0][0]

In [0]:
assert add(9, 0) == 1
assert add(14, 2) == 0 
assert add(15, 7) == 0